<a href="https://colab.research.google.com/github/sgr1118/PyTorch/blob/main/Chapter16_%ED%91%9C%ED%98%84%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 16.1 특징(feature)이란?
- 어떻게 딥러닝이 데이터를 학습하는지 이해하기 위해서 특징의 개념에 알아야 한다. 앞에서는 딥러닝을 활용하여 데이터를 학습하고 평가하는 가장 기초적인 방법을 배웠다면 이제부터는 지금까지 배운 내용을 바탕으로 왜 학습이 잘 되는지에 대해 알아간다.

### 1. 몽타주
- 몽타주는 범죄자를 검거하기 위하여 목격자로부터 나온 진술을 바탕으로 만들어진다. 여기서 좋은 조건과 나쁜 조건이있다. 나쁜 조건일수록 두루뭉실하고 좋은 조건일수록 그 범죄자를 구분짓기 명확하다.


![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/2-01/01-montage.png)
<center>실제 몽타주</center>

### 2. 특징
- 모댈은 원하는 출력 결과를 도출하는 입력 샘플을 다른 샘플들과 구분해낼 수 있는 능력이 필요하다. 따라서 데이터의 샘플을 잘 나타내는 특징을 추출하고 학습할 수 있어야 한다. 어떤 사람을 특정하기 위하여 아래와 같은 예시가 있다고치자

- 연속 값 : 나이, 키, 몸무게, 소득
- 카테고리 값 : 성별, 직업, 거주지, 출신 학교/학과

- 이런 특징 값으로 여러명의 사람들의 데이터가 존재할 때는 비슷한 특징을 가진 사람끼리 묶을 수 있다. ML에서는 어떤 특징을 바탕으로 샘플을 구분할 수 있는지 알아보자. 앞에서 사용한 MNIST를 예를 들어본다면 (1) 곧은 선과 위러진 선이 얼마나 있는가?, (2) 곧은 선과 휘어진 선들이 서로 어떻게 이어져 있는가? 여기서 더 나아가서 (1) 특정 위치의 선이 얼마나 굵은가?, (2) 특정 위치의 선이 얼마나 기울어져 있는가?

- 앞과 같은 특징들이 샘플 구분에 용이하다는 가정하에 각 샘플별로 특징을 수치화해서 모델에 제공한다면 숫자를 분류하거나 샘플을 구분하는데 유용하게 활용될 것이다. 만약 앞에서 제시한 가설이 틀렸다면 모델은 잘 구분할 수 없을 것이다. 이때 기존 ML과 딥러닝 수행 방법의 차이를 확연히 알 수 있다.

### 3. 특징 추출 방법 : ML Vs 딥러닝
- 전통적인 ML에서는 주로 사람이 직접 가정을 세우고 특징을 추출하는 방법을 설정한다. 딥러닝은 Raw한 데이터를 모델에 넣어 모델이 스스로 특징을 파악하고 추출하는 과정을 거쳐 분류 또는 회귀를 수행한다.

||ML|딥러닝|
|-|-|-|
|장점|사용자가 직접 세운 가설을 통한 추출이므로 모델의 동작 및 결과 해석이 쉽다.|모델이 스스로 학습하여 추출하기 때문에 사용자가 미처 몰랐던 특징들도 활용이 가능하다.|
|단점|잘못된 가설일 경우 미처 생각지 못한 특징이 존재할 수 있다.|모델 내부적으로 스스로 추출하고 가고안 특징을 활용하기 때문에 이를 해석하기 어렵다. 모델의 성능이 만족스럽지 않을 때 원인을 분석하는데 어려움이 있다.|

### 4. 특징 벡터
- 특징 벡터는 특징들을 모아서 벡터로 표현하는 것이다. 엑셀을 보면 각 행마다 특징별로 값을 기록하는데 특징 벡터의 예시라고 볼 수 있다. 특징 벡터를 구하면 이를 통해 샘플 사이의 거리(유사도)를 구할 수 있다.

## 16.2 원 핫 인코딩

### 1. 연속 vs 카테고리 값
- 우리가 다루는 데이터는 대부분 연속 값과 카테고리 값들로 구성되어있다. 연속 값은 비슷하 값이라면 서로 비슷하다는 의미를 지니지만, 카테고리 값은 비슷할 지라도 서로 상관없다는 의미를 지닌다. 아래 예제를 보자.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/2-01/02-word_map.png)
<center></center>

- 현실에서는 볼펜과 연필의 관계가 연필과 자의 관계보다 가깝다고 인식하고있다. 
- $distance(연필, 볼펜) < distance(연필, 자)$
- 하지만 임의로 부연된 인덱스 값을 놓고 봤을 때 인덱스를 활용하여 계산한 거리는 우리의 상식과는 다름을 보여준다.
- $|연필 - 볼펜| = 4 > |연필 - 자| = 1$ 따라서 우리는 카테고리 값을 특징 벡터로 표현할 때에는 인덱스 값으로 표기하는 대신 다른 방법을 선택해야한다. 만약 그대로 인게스 값으로 표현하면 코사인 유사도나 유클리디안 거리에서 이상한 계산이 이루어질 수 있다.

### 2. 원 핫 인코딩
- 위에서 발생한 문제를 해소하고자 하는 방법이 원 핫 인코딩이다. 크기가 의미를 갖는 정부로 나타내는 대신 1과 n-1개의 0으로 이루어진 n차원의 벡터로 나타내는 방법을 의미한다.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/2-01/02-map_to_one-hot.png)
<center>n개의 항목 -> n차원</center>

- 위 그림처럼 구성된 원 핫 벡터들은 서로 직교한다. 또한 서로 다른 두 벡터 사이의 코사인 유사도는 항상 0이며 유클리디안 거리는 $sqrt{2}$이다.

- 이처럼 벡터 대부분의 차원이 0인 경우를 희소 벡터라고 부르고 이와 반대되는 개념을 고밀도 벡터라고한다.

### 3. 단어 임베딩
- NLP는 기본적으로 단어를 다룬다. 따라서 모델에 입력 시 원 핫 인코딩을 사용하게 된다. 하지만 단어의 개수가 매우 많기때문에 차원이 커져서 비효율적이고 원 핫 인코딩을 통해서는 단어 사이의 유사도를 표현할 수 없다. 이때 필요한 것이 단어 임베딩이다. 우린 단어 인베딩 기법을 통해 원 핫 벡터로 표현된 단어를 고밀도 벡터로 표현할 수 있게 된다. 다음 그림은 고밀도 벡터로 표현된 단어의 예시이다.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/2-01/02-word_embedding.png)
<center>고밀도 벡터로 표현된 단어</center>

## 16.3 차원 축소
- 앞서 희소 벡터의 특징을 알아보았다. 우리가 사용한 MNIST 데이터도 희소 벡터의 성질을 일부 가지고있다.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/2-01/03-mnist.png)
<center>희소 벡터의 성질을 지닌 일부 MNIST의 샘플</center>

- 그림을 보면 글씨가 존재하는 부분은 255에 가까운 값을 가지고 흰 바탕은 0으로 표시되어있다. 다시말하면 MNIST 데이터의 샘플들이 존재하는  28 * 28 = 784 차원 공간의 대부분은 비어있고, 일부 영역에만 샘플이 존재한다. 이때 굳이 사용할 필요가 없는 샘플들이 있음을 알 수 있다.

### 1. 선형 차원 축소 : 주성분분석(PCA)
- 앞서 언급한 문제를 해소하기 위하여 사용하는 방법이 PCA이다. 고차원의 공간에 샘플들이 분포하고 있을 때 분포를 잘 설명하는 새로운 축(axis)을 찾아내는 과정이다. 이를 통해 선쳥 차원 축소를 수행할 수 있다. 차원 축소를 수행하면 앞과 같이 희소한 벡터들이 존재할 때 좀 더 밀노 높은 표현의 벡터로 나타낼 수 있게 된다.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/2-01/03-pca.png)
<center>PCA 적용</center>

- 다음 그림과 같이, 주어진 샘플들에 대해 두 가지 조건을 만족하는 축을 찾아 해당 샘플에 투사하면 낮은 차원의 샘플로 변환이 가능하다.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/2-01/03-pca_condition.png)
<center>낮은 차원의 샘플로 변환</center>

- 새롭게 찾아낸 축에 투사된 샘플들의 분산이 클수록 샘플들 사이의 특징을 잘 표현할 수 있으며, 투사하는 거리가 작아질수록 잃어버리는 정보가 줄어들게 된다. PCA를 통해 성공적인 차원 축소를 수행하면 다음의 그림과 같이 두 그룹을 분리하는 경계선도 찾아낼 수 있을 것이다.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/2-01/03-binary_classification.png)
<center>두 그룹을 분리하는 경계선 확인</center>

- 다만 샘플이 비선형적인 경계선을 갖는 경우 PCA를 수행하기 어렵다. 이런 경우 비선형 차우너 축소를 도입해야한다.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/2-01/03-nonlinear_binary_classification.png)
<center>비선형적인 경계선을 가진 경우</center>




## 16.4 오토인코더

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/2-01/04-ae.png)
<center>오토인코더</center>

- 오토인코더는 비선형 차원 축소를 수행할 있도록 하는 DNN이다. 오토 인코더는 인코더, 디코더로 구성되어있다. 인코더는 데이터의 차원 축소와 복원을 수행한다. 인코더는 최대한 입력 샘플의 정보를 보존하도록 손실 압축을 진행하고, 디코더는 인코더의 중간 결과물을 받아 입력 샘플과 같아지도록 압축 해제를 수행한다.

- 결과적으로 복원을 성공적으로 하기 위해서 오토인코더는 학습 데이터의 특징을 추출하는 방법을 자동으로 학습하게된다. 오토인코더는 입력 샘플 x와 출력 벡터 $\hat{x}$의 MSE 손실을 최소화하도록 학습한다.

### 1. 인코더와 디코더의 역할
- 인코더는 입력 받은 고차원 데이터를 손실 압축을 한다. 위에서 인코더가 뱉어낸 z 값은 입력 샘플보다 낮은 차원을 가진다. 만약 정보량에 비하여 차원이 크다면 손실되는 정보가 적고 그 반대라면 손실되는 정보가 많아진다. 인코더는 효율적인 압축을 위하여 쓸모없는 정보는 버릴 것이다. 예를들어 MNIST데이터를 보면 흰색 바탕 픽셀에 대한 정보는 학습에 불 필요하므로 이런 정보를 버릴 것이다.

- 인코더의 중간 결과문 z는 입력 샘플을 최대한 보존하고있다. 따라서 디코더는 주어진 z 벡터를 활용하여 입력 샘플을 복원할 수 있을것이다. 이 과정에서 z 벡터는 효율적인 압축을 수행해야 하므로 고밀도 벡터로 표현된다. 또한 z 벡터 또한 입력 샘플에 대한 특징 벡터라고 볼 수 있다.

### 2. 은닉 표현
- 이렇게 얻어진 z 벡터를 입력 샘플에 대한 은닉 표현이라고한다. 그리고 해당 벡터가 존재하는 공간을 은닉 공간이라고 부른다. 아래는 MNIST로 표현한 데이터셋 은닉 공간이다.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/2-01/04-latent_space.png)
<center>은닉 공간에 표현된 MNIST</center>

- 기존 특징 벡터들은 각 차원별로 수치가 의미하는 것이있었다. MNIST처럼 첫 번째 차원은 좌상단 첫 번째 픽셀의 글씨 존재 유무를 수치로 표현한 것이라고 봐도 될 것이다. 하지만 은닉 벡터의 각 차원이 무엇을 의미하는지 알 수 없다. 그래서 은닉이라는 이름이 붙여진 것이다. 다만 앞의 그림처럼 각 차원이 의미하는 바는 해석할 수 없지만 비슷한 특징을 갖는 샘플들이 비슷한 값을 갖는 것을 알 수 있다.

- 은닉 벡터는 원하는 출력을 뱉어내는 데 필요한 정보를 해당 모델만 알고 있는 방법으로 감춰놓고 있습니다. 이것은 꼭 오토인코더가와 같이 입력 샘플을 똑같이 복원하는 데에만 적용되는 것이 아니라 분류와 같은 문제에도 적용됩니다.

- 예를 들어 MNIST 숫자를 분류한다는 관점에서도 필요한 정보는 분명히 존재할 것이다. 그렇다면 MNIST 분류 모델 중간 계층의 중간 결과물 벡터는 분류에 필요한 정보를 꼭 담고 있어야한다. 비록 분류를 위한 정보는 샘플을 똑같이 복원을 위한 정보다 훨씬 적을 것이다. 그리고 해당 중간 결과물 벡터는 아마도 입력 샘플보다 훨씬 작은 차원으로 표현되고 있을 것이다. 따라서 해당 중간 결과물 벡터도 당연히 입력 샘플에 대한 특징 벡터라고 볼 수 있으며 은닉 벡터라고도 할 수 있다. 아래 그림은 그걸 표현한다.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/2-01/04-hidden_vectors.png)
<center>은닉 벡터</center>

- 위 그림처럼 3개 계층이 존재하고 중간에 은닉 표현 $h_1, h_2$이 있다. 입력 벡터 x의 차원 크기에 비하여 은닉 표현이 출력에 가까워 질수록 더 작아지는 것을 볼 수 있다. 이처럼 입력에 비해 더 작거나 비슷한 차원을 활용하여 데이터의 비선형 관계를 풀어내도록 모델의 구조를 설계한다. 따라서 오토인코더의 병목 구간처럼 x -> y 관계를 예측하는데 필요한 정보를 추출할 수 있도록 신경망의 중간 계층들이 학습될 것입니다. 

## 16.5 마치며

### 요약

1. 특징
 - 좋은 특징이란 모델이 원하는 출력 결과를 뱉어내기 위하여 해당 출력을 지니는 입력 샘플들을 다른 샘플들로부터 구분해낼 수 있는 특징을 말함
 - 모델은 데이터의 샘플을 잘 설명하는 특징을 추출하고 학습할 수 있어야함
 - 기존 ML은 사용자가 직접 가설을 세우도 직접 뽑아낸 특징(hand-crafted feature)을 활용하여 모델이 패턴을 인식
 - 딥러닝은 rar data에서 모델이 직접 특징을 추출하여 학습을 수행

2. 원 핫 인코딩
 - 연속 값과 카테고리 값의 결정적인 차이점으로 연속 값은 비슷한 값이라면 서로 비슷하다는 의미를 지니지만 카테고리 값은 비슷한 값일지라도 서로 상관없다는 의미를 가짐
 - 원 핫 인코딩은 카테고리 값을 표현하는 방법
 - 크기가 의미를 갖는 정수로 나타내는 방법, 한 개의 1과 n-1개의 0으로 이루어진 n차원의 벡터를 통해 표현

3. 은닉 표현
 - 잠재 표현이라고 부르기도 함
 - DNN의 내부 계층이 데이터를 잘 설명할 수 있도록 출력 차원인 저차원의 공간으로 비선형 변환을 하는 과정에서 얻어진 결과물
 - 비슷한 성격의 샘플들을 비슷한 은닉 표현을 가지고 있을 가능성이 높음
 - 은닉 표현이 어떠한 의미를 지니는지 해석하는 것은 어려움